In [191]:
from PIL import Image
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

import os
import cv2 as cv
import glob
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [192]:
CAPTCHA_CHARSET = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9','a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z', ]   # 驗證碼字符集
CAPTCHA_LEN = 4            # 驗證碼長度
CAPTCHA_HEIGHT =  35            # 驗證碼高度
CAPTCHA_WIDTH =   68           # 驗證碼寬度

TRAIN_DATA_DIR = 'datas/train/' # 驗證碼數據集目錄
TEST_DATA_DIR = 'datas/test/' # 數據集目錄

BATCH_SIZE = 100
EPOCHS = 100
OPT = 'adam'
LOSS = 'binary_crossentropy'

MODEL_DIR = 'models/train_demo/'
MODEL_FORMAT = '.h5'
HISTORY_DIR = 'models/history/'
HISTORY_FORMAT = '.history'

filename_str = "{}captcha_{}_{}_bs_{}_epochs_{}{}"

# 模型網絡結構文件
MODEL_VIS_FILE = 'captcha_classfication' + '.png'
# 模型文件
MODEL_FILE = filename_str.format(MODEL_DIR, OPT, LOSS, str(BATCH_SIZE), str(EPOCHS), MODEL_FORMAT)
# 訓練記錄文件
HISTORY_FILE = filename_str.format(HISTORY_DIR, OPT, LOSS, str(BATCH_SIZE), str(EPOCHS), HISTORY_FORMAT)

In [193]:
def rgb2gray(img):
    # Y' = 0.299 R + 0.587 G + 0.114 B 
    return np.dot(img[...,:3], [0.299, 0.587, 0.114])

In [194]:
# 定義one-hot編碼函數

def text2vec(text,length=CAPTCHA_LEN,charset=CAPTCHA_CHARSET):
    text_len = len(text)
    # 驗證碼長度校驗
    if text_len != length:
        raise ValueError("Error:length of captcha should be{},but got {}".format(length,text_len))
    # 生成一個形如(CAPTCHA_LEN*CAPTCHA_CHARSET)的一維向量
    # 例如，4個純數字的驗證碼生成形如(4*10,)的一維向量
    vec = np.zeros(length*len(charset)) #生成一个默认为0的向量
    for i in range(length):
        # One-hot編碼驗證碼中的每個數字
        # 每個字符的熱碼 = 索引 +偏移量
        vec[charset.index(text[i]) + i*len(charset)] = 1
    return vec



In [195]:
# 向量转回文本

def vec2text(vector):
    if not isinstance(vector, np.ndarray):
        vector = np.asarray(vector)
    vector = np.reshape(vector, [CAPTCHA_LEN, -1])
    text = ''
    for item in vector:
        text += CAPTCHA_CHARSET[np.argmax(item)]
    return text

In [196]:
def fit_keras_channels(batch, rows=CAPTCHA_HEIGHT, cols=CAPTCHA_WIDTH):
    if K.image_data_format() == 'channels_first':
        batch = batch.reshape(batch.shape[0], 1, rows, cols)
        input_shape = (1, rows, cols)
    else:
        batch = batch.reshape(batch.shape[0], rows, cols, 1)
        input_shape = (rows, cols, 1)
    
    return batch, input_shape

In [198]:
X_train = []
Y_train = []
Y_filename = []
for filename in glob.glob(TRAIN_DATA_DIR + "*.jpg"):
    X_train.append(np.array(Image.open(filename)))
    Y_filename.append(filename)
#    Y_train.append(filename.lstrip("datas/train\\").rstrip(".png"))
for filename in glob.glob(TRAIN_DATA_DIR + "*.png"):
    X_train.append(np.array(Image.open(filename)))
    Y_filename.append(filename)

In [199]:
# list -> rgb(numpy)
X_train = np.array(X_train, dtype=np.float32)
# rgb -> gray
X_train = rgb2gray(X_train)
# normalize
X_train = X_train / 255
# Fit keras channels
X_train, input_shape = fit_keras_channels(X_train)

print(X_train.shape, type(X_train))
print(input_shape)

(7530, 35, 68, 1) <class 'numpy.ndarray'>
(35, 68, 1)


In [203]:
for i in range(len(Y_filename)):
    Y_train.append(Y_filename[i][12:16])
for i in range(len(Y_train)):
    Y_train[i] = text2vec(Y_train[i])

In [206]:
Y_train = np.asarray(Y_train)
print(Y_train.shape, type(Y_train))
print(Y_train[0])

(7530, 144) <class 'numpy.ndarray'>
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [210]:
Y_train.shape

(7530, 144)

In [211]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)

In [212]:
X_train.shape, X_val.shape, Y_train.shape, Y_val.shape

((6777, 35, 68, 1), (753, 35, 68, 1), (6777, 144), (753, 144))

In [221]:
load_model_name = "0806_2"
load_model_path = './models/{}.h5'.format(load_model_name)
model_name = "0806_3"
model_path = './models/{}.h5'.format(model_name)

In [222]:
try:
    model = load_model(load_model_path)
except Exception as e:
    print('#######Exception', e)
    model = crack_captcha_cnn()

In [215]:
# 輸入層
inputs = Input(shape = input_shape,name = "inputs")


# 第1層卷積
conv1 = Conv2D(32,(3,3),name="conv1")(inputs)
relu1 = Activation("relu",name="relu1")(conv1)

# 第2層卷積
conv2 = Conv2D(128,(5,5),name ="conv2")(relu1)
relu2 = Activation("relu",name="relu2")(conv2)
pool2 = MaxPooling2D(pool_size=(2,2),padding="same",name="pool2")(relu2)

# 第3層卷積
conv3 = Conv2D(256,(5,5),name="conv3")(pool2)
relu3 = Activation("relu",name="relu3")(conv3)
pool3 = MaxPooling2D(pool_size=(2,2),padding="same",name="pool3")(relu3)

# 將Pooled feature map 攤平後輸入全連接網絡
x = Flatten()(pool3)

# Dropout
#x = Dropout(0.25)(x)

# 4個全連接層分別做10分類，分別對應4個字符
x = [Dense(36,activation="softmax",name="fc%d"%(i+1))(x) for i in range(4)]

# 4個字符向量拼接在一起，與標籤向量形式一致，作爲模型輸出
outs = Concatenate()(x)

# 定義模型的輸入與輸出
model = Model(inputs=inputs,outputs=outs)
optimizer = keras.optimizers.Adam(lr=10e-6)
model.compile(#loss=LOSS,
    loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

In [216]:
print(model.summary())

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 35, 68, 1)]  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 33, 66, 32)   320         inputs[0][0]                     
__________________________________________________________________________________________________
relu1 (Activation)              (None, 33, 66, 32)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 29, 62, 128)  102528      relu1[0][0]                      
____________________________________________________________________________________________

In [217]:
plot_model(model,show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [225]:
datagen = ImageDataGenerator(
#    rotation_range=0.01,
    width_shift_range=0.01,
    height_shift_range=0.01,
    shear_range=0.001,
#    zoom_range=0.2,
    horizontal_flip=False,
    channel_shift_range=10,
#    cval=0,
#    zca_epsilon=1e-6,
    fill_mode='nearest'
)



optimizer = keras.optimizers.Adam(lr=10e-4)

model_path = './models/{}.h5'.format(model_name)
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

model.compile(optimizer=optimizer, 
#              loss='categorical_crossentropy',
              loss= LOSS,
              
              metrics=['accuracy'])


batch_size = 16
epochs = 200

history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size = batch_size),
                              epochs = epochs,
                              validation_data = (X_val, Y_val),
                             callbacks = [checkpoint, earlystop]
                             )

Epoch 1/200
423/424 [============================>.] - ETA: 0s - loss: 0.1119 - accuracy: 0.9737
Epoch 00001: val_loss improved from inf to 0.02852, saving model to ./models/0806_3.h5
424/424 [==============================] - 226s 533ms/step - loss: 0.1120 - accuracy: 0.9737 - val_loss: 0.0285 - val_accuracy: 0.9920
Epoch 2/200
423/424 [============================>.] - ETA: 0s - loss: 0.1115 - accuracy: 0.9738
Epoch 00002: val_loss improved from 0.02852 to 0.02610, saving model to ./models/0806_3.h5
424/424 [==============================] - 226s 533ms/step - loss: 0.1115 - accuracy: 0.9738 - val_loss: 0.0261 - val_accuracy: 0.9918
Epoch 3/200
423/424 [============================>.] - ETA: 0s - loss: 0.1118 - accuracy: 0.9738
Epoch 00003: val_loss did not improve from 0.02610
424/424 [==============================] - 226s 532ms/step - loss: 0.1118 - accuracy: 0.9738 - val_loss: 0.0278 - val_accuracy: 0.9920
Epoch 4/200
423/424 [============================>.] - ETA: 0s - loss: 0.11

In [ ]:
model_history = history

training_loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.plot(training_loss, label="training_loss")
plt.plot(val_loss, label="validation_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend(loc='best')
plt.show()

In [ ]:
training_acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']

plt.plot(training_acc, label="training_acc")
plt.plot(val_acc, label="validation_acc")
plt.xlabel("Epochs")
plt.ylabel("Acc")
plt.title("Learning Curve")
plt.legend(loc='best')
plt.show()

In [ ]:
model = load_model(model_path)

scores = model.evaluate(X_test, Y_test, verbose=1)
print('Validation loss:', scores[0])
print('Validation accuracy:', scores[1])


### Test

In [85]:
X_test = []
Y_test = []
Y_testfilename = []
for filename in glob.glob(TEST_DATA_DIR + "*.jpg"):
    X_test.append(np.array(Image.open(filename)))
    Y_testfilename.append(filename)
#    Y_train.append(filename.lstrip("datas/train\\").rstrip(".png"))
for filename in glob.glob(TEST_DATA_DIR + "*.png"):
    X_test.append(np.array(Image.open(filename)))
    Y_testfilename.append(filename)

In [86]:
# list -> rgb(numpy)
X_test = np.array(X_test, dtype=np.float32)
# rgb -> gray
X_test = rgb2gray(X_test)
# normalize
X_test = X_test / 255
# Fit keras channels
X_test, input_shape = fit_keras_channels(X_test)

print(X_test.shape, type(X_test))
print(input_shape)

(426, 35, 68, 1) <class 'numpy.ndarray'>
(35, 68, 1)


In [90]:
for i in range(len(Y_testfilename)):
    Y_test.append(Y_testfilename[i][11:15])

In [92]:
for i in range(len(Y_test)):
    Y_test[i] = text2vec(Y_test[i])

In [93]:
Y_test = np.asarray(Y_test)
print(Y_test.shape, type(Y_test))
print(Y_test[0])

(426, 144) <class 'numpy.ndarray'>
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [95]:
vec2text(Y_test[20])

'3adc'

In [98]:
X_test = X_test
X_id = Y_test

model_name = "0806_2"
model_path = './models/{}.h5'.format(model_name)
model = load_model(model_path)


In [69]:
y_test_pred_prob = model.predict(X_test)

In [70]:
y_test_pred_prob

array([[7.5220783e-22, 6.3255320e-22, 9.9980253e-01, ..., 7.7217908e-07,
        2.8623782e-07, 3.6055965e-08],
       [4.2497086e-18, 2.7107968e-18, 9.8203421e-01, ..., 7.9522779e-09,
        2.2685040e-10, 4.1856758e-05],
       [3.2262162e-22, 3.4298120e-22, 9.9998415e-01, ..., 2.9481944e-07,
        6.3760645e-08, 2.8121911e-13],
       ...,
       [5.7013243e-23, 7.5167539e-23, 1.9867988e-03, ..., 3.1421424e-11,
        1.3047622e-05, 2.9318370e-20],
       [6.9136401e-18, 6.5181443e-18, 4.8938729e-03, ..., 3.7162674e-06,
        9.4698113e-01, 2.6321171e-11],
       [1.4219442e-20, 1.3783637e-20, 2.9486281e-01, ..., 3.0089883e-05,
        4.5823079e-09, 1.1636355e-06]], dtype=float32)

In [105]:
y_test_pred_prob = model.predict(X_test)
y_test_pred = []
for i in range(len(y_test_pred_prob)):
    y_test_pred.append(vec2text(y_test_pred_prob[i]))
    


In [113]:
Y_ans=[]
for i in range(len(Y_testfilename)):
    Y_ans.append(Y_testfilename[i][11:15])
Y_ans

['237j',
 '24je',
 '25r7',
 '26df',
 '28r5',
 '2a8c',
 '2a8f',
 '2aet',
 '2crr',
 '2g33',
 '2hk2',
 '2r43',
 '2u65',
 '2xa3',
 '2zc2',
 '35jf',
 '36g7',
 '378c',
 '37tr',
 '3959',
 '3adc',
 '3gkf',
 '3hb3',
 '3xyq',
 '448q',
 '47xh',
 '47zh',
 '4a29',
 '4a65',
 '4cse',
 '4dre',
 '4dxs',
 '4e9a',
 '4g2y',
 '4jz6',
 '4ktz',
 '4p6g',
 '4q62',
 '4qdf',
 '4r2z',
 '4rj7',
 '4s2u',
 '4sak',
 '4t4e',
 '548u',
 '577p',
 '58y9',
 '5arj',
 '5epp',
 '5h5y',
 '5qhr',
 '5rjb',
 '5sds',
 '5uzc',
 '5z8h',
 '63kt',
 '63xg',
 '649p',
 '65b4',
 '6754',
 '67db',
 '67e3',
 '68q7',
 '69g2',
 '6c2h',
 '6dgc',
 '6e7h',
 '6ecz',
 '6f5f',
 '6gkf',
 '6kbc',
 '6r9b',
 '6rs8',
 '6y8k',
 '6zrf',
 '7857',
 '78ay',
 '78f2',
 '78sr',
 '7b38',
 '7d8d',
 '7eg3',
 '7g8b',
 '7hkb',
 '7kgr',
 '7kut',
 '7p33',
 '7p7u',
 '7r3t',
 '7sur',
 '7th7',
 '7zes',
 '83gd',
 '83h4',
 '854y',
 '8ak9',
 '8b49',
 '8bzb',
 '8cz7',
 '8ecs',
 '8fhj',
 '8fsd',
 '8h2p',
 '8jkx',
 '8js4',
 '8kdg',
 '8q6a',
 '8q7u',
 '8rrk',
 '8s5t',
 '8u6p',
 

In [109]:
y_test_pred

['237j',
 '24je',
 '25r7',
 'z6dr',
 '29r5',
 '2a8c',
 '2a8f',
 '2aej',
 '2crr',
 '2q33',
 '2hk2',
 '2r43',
 '2u65',
 '2xa3',
 '2zc2',
 '35jf',
 '36g7',
 '378c',
 '377r',
 '3959',
 '3adk',
 '3gkf',
 '3hb3',
 '3xyq',
 'c48q',
 '47xh',
 '47zh',
 '4a29',
 '4a65',
 '4cse',
 '4dje',
 '4dhs',
 '4e9a',
 '4g2y',
 '4jz6',
 '4ktz',
 '4p6g',
 '4q62',
 '4qdf',
 '4r2z',
 '4rj7',
 'fz2u',
 '43ak',
 '4t4e',
 '548u',
 '577p',
 '58y9',
 '5agj',
 '5epp',
 '5hsy',
 '5gyr',
 '5rjb',
 '5sds',
 '5uzc',
 '5z8h',
 '63kt',
 '63ug',
 '649p',
 '65bh',
 '6754',
 '67db',
 '67e3',
 '68q7',
 '69g2',
 '6c2h',
 '6dgc',
 '6e7h',
 '6ecz',
 '6f5f',
 '6gkf',
 '6kbc',
 '6r9b',
 '6rs8',
 '6y8k',
 '6zrf',
 '7857',
 '78ay',
 '78f2',
 '78sr',
 '7ba8',
 '7d8d',
 '7eg3',
 '7gdb',
 '7hkb',
 '7kgr',
 '7kut',
 '7p33',
 '7p7u',
 '7r3t',
 '7eur',
 '7th7',
 '7zes',
 '83gd',
 '83h4',
 '854y',
 '3ak9',
 '8b49',
 '8uzb',
 '8cz7',
 '8ecs',
 '8fhj',
 '3frr',
 '8h2p',
 '8jkx',
 '8js4',
 '8tdg',
 '8g6a',
 '8q7u',
 '8rrk',
 '835t',
 '8u6p',
 

In [116]:
y_test_pred_df = pd.DataFrame({'id': np.array(Y_ans), 'class':y_test_pred}).sort_values(by='id')
y_test_pred_df.to_csv('./submissions/{}.csv'.format(model_name), index=False)